In [1]:
#Packages for Data Processing and Cleaning
import numpy as np # linear algebra
import pandas as pd 
import os

**Loading the Dataset**

In [2]:
df = pd.read_csv('/Users/georgiosskourlis/Documents/DataScience_salaries_2024.csv')

df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,MI,FT,Data Scientist,30400000,CLP,40038,CL,100,CL,L
1,2021,MI,FT,BI Data Analyst,11000000,HUF,36259,HU,50,US,L
2,2020,MI,FT,Data Scientist,11000000,HUF,35735,HU,50,HU,L
3,2021,MI,FT,ML Engineer,8500000,JPY,77364,JP,50,JP,S
4,2022,SE,FT,Lead Machine Learning Engineer,7500000,INR,95386,IN,50,IN,L


**Checking for Nulls**

In [3]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

**Data Overview**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14838 entries, 0 to 14837
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           14838 non-null  int64 
 1   experience_level    14838 non-null  object
 2   employment_type     14838 non-null  object
 3   job_title           14838 non-null  object
 4   salary              14838 non-null  int64 
 5   salary_currency     14838 non-null  object
 6   salary_in_usd       14838 non-null  int64 
 7   employee_residence  14838 non-null  object
 8   remote_ratio        14838 non-null  int64 
 9   company_location    14838 non-null  object
 10  company_size        14838 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.2+ MB


**Creating Job Categories**

In [5]:
# Simplifies job titles to broader job category labels and creates an original column.

# Mapping job titles to job categories.
def map_job_title(df, original_column='job_title', new_column='job_category'):

    # Define the mapping dictionary
    job_title_mapping = {
        'data analyst': 'Data Analyst',
        'bi analyst': 'Data Analyst',
        'analytics': 'Data Analyst',
        'data scientist': 'Data Scientist',
        'data science': 'Data Scientist',
        'decision scientist': 'Data Scientist',
        'data engineer': 'Data Engineer',
        'etl': 'Data Engineer',
        'pipeline': 'Data Engineer',
        'machine learning': 'ML Engineer',
        'ml': 'ML Engineer',
        'deep learning': 'ML Engineer',
        'ai': 'ML Engineer',
        'data architect': 'Data Engineer',
        'big data': 'Data Engineer',
        'cloud data': 'Data Engineer',
        'data manager': 'Data Engineer',
        'head of data': 'Data Engineer',
        'director': 'Data Engineer',
        'bi developer': 'Data Analyst',
        'power bi': 'Data Analyst',
        'business intelligence': 'Data Analyst',
        'research scientist': 'Data Scientist',
        'research engineer': 'Data Scientist',
        'statistician': 'Data Analyst',
        'quantitative': 'Data Analyst',
        'computational biologist': 'Data Scientist',
        'robotics': 'Data Engineer',
        'nlp': 'Data Engineer',
        'applied scientist': 'Data Scientist',
        'research analyst': 'Data Analyst',
        'data specialist ': 'Data Scientist',
        'data modeler': 'Data Engineer',
    }

    # Convert job titles to lowercase and map them to broader categories
    df[new_column] = df[original_column].str.lower().apply(
        lambda title: next((v for k, v in job_title_mapping.items() if k in title), 'Data Scientist')
    )
    return df

# Example usage
df = map_job_title(df)

**Output**

In [6]:
df['job_category']

0        Data Scientist
1          Data Analyst
2        Data Scientist
3           ML Engineer
4           ML Engineer
              ...      
14833      Data Analyst
14834      Data Analyst
14835       ML Engineer
14836      Data Analyst
14837       ML Engineer
Name: job_category, Length: 14838, dtype: object

**Redefining Work Models**

In [7]:
# Converts remote ratio to work models labels and creates an original column.

# Mapping remote ratio to work models.
def map_remote_ratio(df, original_column='remote_ratio', new_column='work_model'):

    # Define the mapping dictionary
    remote_ratio_mapping = {
        0: 'In-Person',
        50: 'Hybrid',
        100: 'Remote'
    }

    # Apply the mapping with a default value of 'Other' for unmapped numbers
    df[new_column] = df[original_column].map(remote_ratio_mapping).fillna('Other')

    return df

# Example usage
df = map_remote_ratio(df)

**Output**

In [8]:
df['work_model']

0           Remote
1           Hybrid
2           Hybrid
3           Hybrid
4           Hybrid
           ...    
14833       Remote
14834    In-Person
14835       Remote
14836    In-Person
14837       Remote
Name: work_model, Length: 14838, dtype: object

**Categorizing Experience Level**

In [9]:
# Converts experience levels into more descriptive names.

# Mapping experience levels to more descriptive labels.
def map_experience_level(df, original_column='experience_level', new_column='generalized_experience_level'):

    # Define the mapping dictionary
    experience_level_mapping = {
        'EN': 'Junior',
        'MI': 'Mid-Level',
        'SE': 'Senior',
        'EX': 'Expert'
    }

    # Create the new column
    df[new_column] = df[original_column].replace(experience_level_mapping)
    return df

# Example usage
df = map_experience_level(df)

**Output**

In [10]:
df['generalized_experience_level']

0        Mid-Level
1        Mid-Level
2        Mid-Level
3        Mid-Level
4           Senior
           ...    
14833    Mid-Level
14834       Expert
14835       Junior
14836       Junior
14837       Junior
Name: generalized_experience_level, Length: 14838, dtype: object

**Checking for Distinct Employment Types**

In [11]:
grouped_counts = df.groupby('employment_type').size()
grouped_counts

employment_type
CT       26
FL       13
FT    14772
PT       27
dtype: int64

**Converting Employment Types**

In [12]:
#Converting Acronyms to Full Employment Type status
def convert_status(value):
    if value == 'FT':
        return 'Full Time'
    elif value == 'PT':
        return 'Part Time'
    elif value == 'CT':
        return 'Contract'
    elif value == 'FL':
        return 'Freelance'
    else:
        return value  # Keeps other values unchanged

df['employment_type'] = df['employment_type'].apply(convert_status)

**Output**

In [13]:
df['employment_type']

0        Full Time
1        Full Time
2        Full Time
3        Full Time
4        Full Time
           ...    
14833    Full Time
14834    Full Time
14835    Full Time
14836    Full Time
14837    Part Time
Name: employment_type, Length: 14838, dtype: object

**Checking For Negative Values**

In [14]:

undesired_values = df[df['salary'] <= 0].sort_values('salary', ascending=False).count()

undesired_values_df = undesired_values.reset_index()

undesired_values_df.columns = ['Salary', 'Count']

undesired_values_df

,Salary,Count
0,work_year,0
1,experience_level,0
2,employment_type,0
3,job_title,0
4,salary,0
5,salary_currency,0
6,salary_in_usd,0
7,employee_residence,0
8,remote_ratio,0
9,company_location,0


**Data Frame Ready for Analysis**

In [15]:
df.head(50)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_category,work_model,generalized_experience_level
0,2021,MI,Full Time,Data Scientist,30400000,CLP,40038,CL,100,CL,L,Data Scientist,Remote,Mid-Level
1,2021,MI,Full Time,BI Data Analyst,11000000,HUF,36259,HU,50,US,L,Data Analyst,Hybrid,Mid-Level
2,2020,MI,Full Time,Data Scientist,11000000,HUF,35735,HU,50,HU,L,Data Scientist,Hybrid,Mid-Level
3,2021,MI,Full Time,ML Engineer,8500000,JPY,77364,JP,50,JP,S,ML Engineer,Hybrid,Mid-Level
4,2022,SE,Full Time,Lead Machine Learning Engineer,7500000,INR,95386,IN,50,IN,L,ML Engineer,Hybrid,Senior
5,2021,MI,Full Time,ML Engineer,7000000,JPY,63711,JP,50,JP,S,ML Engineer,Hybrid,Mid-Level
6,2021,SE,Full Time,Data Science Manager,7000000,INR,94665,IN,50,IN,L,Data Scientist,Hybrid,Senior
7,2022,EN,Full Time,Data Scientist,6600000,HUF,17684,HU,100,HU,M,Data Scientist,Remote,Junior
8,2022,EX,Full Time,Head of Machine Learning,6000000,INR,76309,IN,50,IN,L,ML Engineer,Hybrid,Expert
9,2022,EN,Full Time,Research Engineer,5500000,JPY,41809,JP,50,JP,L,Data Scientist,Hybrid,Junior
